In [1]:
import pandas as pd

**PreReq Detect Demo**

Here we will show off the basic idea behind the PreReq Detect application and how it would be used by a student at the U of I.
Let us first start with the the basic setup of the app.

**Data Ingestion**

At the current time, we do not have access to the data where these minor and certificate requirements are stored. In the mean time, we will create a sample of how the data may be stored in order to test out our concept

In [2]:
#For now, not listing all elective courses to save space. In future, may instead alter data to instead determine class options based on excluded courses.
#Additionally, some other standard of how these courses are stored will need to change based on manner of different minors and certificates are listed.

requirements = pd.DataFrame({
                             'title' : ['minor_cs'],
                             'required_courses' : [['cs_124', 'cs_128', 'cs_173', 'cs_225']],
                             'elective_courses' : [['cs_307', 'cs_340', 'cs_341', 'cs_357', 'cs_361', 'cs_374', 'cs_407', 'cs_409']],
                             'elective_num_required' : [2]
})

In [3]:
display(requirements)

,title,required_courses,elective_courses,elective_num_required
0,minor_cs,"[cs_124, cs_128, cs_173, cs_225]","[cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2


We now will generate some potential students and see if we can use them to determine if they have any of the requirements needed to qualify at all for the listed minors/certificates.

In [4]:
#Completed required courses
student_1 = ['cs_124', 'cs_128', 'cs_173', 'cs_225']

#Completed enough elective courses
student_2 = ['cs_307', 'cs_340', 'cs_341']

#Completed some required and some elective courses
student_3 = ['cs_124', 'cs_307']

#Completed neither any required or elective courses
student_4 = ['stat_107']

We create this function to be used as a checker for both the required and elective course lists. 

In [14]:
def req_finder(requirements, student):
    '''
        Input:
            - requirements: DataFrame, contains each minor/certificate and the associated requirements needed to achieve them.
            - student: List, contains list of all courses completed by a given student
        Output:
            - returns DataFrame containing minors/certificates the student has completed some number of needed courses from.
        Purpose:
            - req_finder returns only the minors/certificates the student has any amount of classes completed for.
    '''
    filtered_requirements = requirements[requirements['required_courses'].apply(lambda x: any(item in student for item in x))]
    filtered_electives = requirements[requirements['elective_courses'].apply(lambda x: any(item in student for item in x))]    
    
    filtered_courses = pd.concat([filtered_requirements, filtered_electives])

    filtered_courses['required_courses'] = filtered_courses['required_courses'].apply(tuple)
    filtered_courses['elective_courses'] = filtered_courses['elective_courses'].apply(tuple)

    qualifying_programs = filtered_courses.drop_duplicates()

    #qualifying_programs['required_courses'] = qualifying_programs['required_courses'].apply(list)
    #qualifying_programs['elective_courses'] = qualifying_programs['elective_courses'].apply(list)

    return qualifying_programs

In [6]:
test1 = req_finder(requirements, student_1)
display(test1)

,title,required_courses,elective_courses,elective_num_required
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2


In [7]:
test2 = req_finder(requirements, student_2)
display(test2)

,title,required_courses,elective_courses,elective_num_required
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2


In [8]:
test3 = req_finder(requirements, student_3)
display(test3)

,title,required_courses,elective_courses,elective_num_required
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2


In [9]:
test4 = req_finder(requirements, student_4)
display(test4)

,title,required_courses,elective_courses,elective_num_required


Using the req_finder, we now can see the minors/certificates relevant to a student's currently completed coursework. We now will try and create separation between the courses they have completed and what remains for required courses and elective courses. We will use student_3 to test this since they have completed a limited amount in both the required course and elective course columns.

In [26]:
student_3_courses = req_finder(requirements, student_3)
display(student_3_courses)

,title,required_courses,elective_courses,elective_num_required
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2


We create this function to separate between classes completed toward the given minor/certificate from those that still would need to be completed.

In [29]:
def completion_separator(student_courses, student):
    '''
        Input:
            - student_courses: DataFrame, contains each minor/certificate and the associated requirements needed to achieve them for minors/certificates the student qualifies for.
            - student: List, contains list of all courses completed by a given student
        Output:
            - returns DataFrame that separates which courses for minor/certificate have been completed versus those remaining
        Purpose:
            - completion_separator returns separation of remaining classes versus still needed classes for visualization purposes
    '''
    student_courses['completed_required_courses'] = student_courses['required_courses'].apply(lambda x: tuple(value for value in x if value in student))
    student_courses['completed_elective_courses'] = student_courses['elective_courses'].apply(lambda x: tuple(value for value in x if value in student))

    student_courses['remaining_required_courses'] = student_courses['required_courses'].apply(lambda x: tuple(value for value in x if value not in student))
    student_courses['remaining_elective_courses'] = student_courses['elective_courses'].apply(lambda x: tuple(value for value in x if value not in student))


    student_courses['elective_num_completed'] = len(student_courses['completed_elective_courses'])
    student_courses['num_elective_remaining'] = student_courses['elective_num_required'] - student_courses['elective_num_completed']
    student_courses['num_elective_remaining'] = student_courses['num_elective_remaining'].apply(lambda x: max(x, 0))

    student_courses = student_courses.drop(columns={'required_courses', 'elective_courses', 'elective_num_required'})

    return student_courses

In [28]:
separation_3 = completion_separator(student_3_courses, student_3)
display(separation_3)

,title,completed_required_courses,completed_elective_courses,remaining_required_courses,remaining_elective_courses,elective_num_completed,num_elective_remaining
0,minor_cs,"(cs_124,)","(cs_307,)","(cs_128, cs_173, cs_225)","(cs_340, cs_341, cs_357, cs_361, cs_374, cs_40...",1,1
